In [1]:
import json
import socket
from datetime import datetime
import pandas as pd

HOST1, PORT1 = "localhost", 9999
HOST2, PORT2 = "localhost", 9998
commodities = ["corn", "wheat", "soybean", "cotton", "coffee"]

# Register to keep track of buy and sell actions
register = []

# Queue to store observed prices for calculating cumulative sum
price_queue = []

# Window size for cumulative sum calculation (7-day rolling)
WINDOW_SIZE = 7

def buy_sell_strategy(commodity_name, prediction):
    """
    Implement buy and sell strategy based on prediction and cumulative sum of observed prices.
    """
    try:
        observed_price = prediction["Observed"]
        predicted_price = prediction["Next Week Prediction"]
    except KeyError:
        print("Received prediction data is not in the expected format.")
        return None
    
    # Add observed price to the queue
    price_queue.append(observed_price)
    
    # Maintain window size for observed prices
    if len(price_queue) > WINDOW_SIZE:
        price_queue.pop(0)  # Remove oldest price if window size exceeded
    
    # Calculate cumulative sum of observed prices
    cumulative_sum = sum(price_queue)
    
    print("Observed Prices in Rolling Window:", price_queue)
    print("Cumulative Sum:", cumulative_sum)
    
    if predicted_price > cumulative_sum:
        action = "Buy"
    elif predicted_price < cumulative_sum:
        action = "Sell"
    else:
        action = "Hold"
    
    register.append({
        "Commodity": commodity_name,
        "Action": action,
        "Observed Price": observed_price,
        "Predicted Price": predicted_price,
        "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })

    return action

def fetch_prediction_data(commodity):
    # Create a socket (SOCK_STREAM means a TCP socket)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            # Connect to Server 1 and send data
            sock.connect((HOST1, PORT1))
            sock.sendall(bytes(commodity + "\n", "utf-8"))

            # Receive data from Server 1
            received = str(sock.recv(1024), "utf-8")
            print("Sent:     {}".format(commodity))
            print("Received:", received)

            # Check if received data is in valid JSON format
            try:
                prediction_data = json.loads(received)
            except json.JSONDecodeError:
                print("Invalid JSON format in response.")
                return None

            # Check if received prediction data is in the expected format
            if "Observed" not in prediction_data or "Next Week Prediction" not in prediction_data:
                print("Received prediction data is not in the expected format.")
                return None

            return buy_sell_strategy(commodity, prediction_data)
        except ConnectionRefusedError:
            print("Connection refused. Server 1 may not be running or the port is incorrect.")
            return None

# Function to send register data to Server 2
def send_register_data_to_Server2(data):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            sock.connect((HOST2, PORT2))
            # Convert data to JSON format
            data_json = json.dumps(data)
            # Send data to Server 2
            sock.sendall(bytes(data_json + "\n", "utf-8"))
            print("Sent register data to Server 2:")
            print(data_json)
        except ConnectionRefusedError:
            print("Connection refused. Server 2 may not be running or the port is incorrect.")

# Fetch prediction data for all commodities
for commodity in commodities:
    decision = fetch_prediction_data(commodity)
    
    if decision:
        # Create a new socket for communication with Server 2
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock2:
            try:
                # Connect to Server 2
                sock2.connect((HOST2, PORT2))
                # Convert decision to JSON format
                decision_json = json.dumps(decision)
                # Send decision to Server 2
                sock2.sendall(bytes(decision_json + "\n", "utf-8"))
                print("Sent decision to Server 2 for commodity {}: {}".format(commodity, decision))
            except ConnectionRefusedError:
                print("Connection refused. Server 2 may not be running or the port is incorrect.")
    else:
        print("No action taken based on prediction for commodity {}.".format(commodity))

# Send register data to Server 2
send_register_data_to_Server2(register)

# Convert register to DataFrame
df_register = pd.DataFrame(register)

# Print register as a table
print("Register Table:")
df_register


Sent:     corn
Received: {"Reference Date": "2024-03-04", "Observed": 417.5, "Next Week Prediction": 484.46875}
Observed Prices in Rolling Window: [417.5]
Cumulative Sum: 417.5
Sent decision to Server 2 for commodity corn: Buy
Sent:     wheat
Received: {"error": "Invalid JSON format in response"}
Received prediction data is not in the expected format.
No action taken based on prediction for commodity wheat.
Sent:     soybean
Received: {"Reference Date": "2024-03-04", "Observed": 1147.0, "Next Week Prediction": 1275.9825}
Observed Prices in Rolling Window: [417.5, 1147.0]
Cumulative Sum: 1564.5
Sent decision to Server 2 for commodity soybean: Sell
Sent:     cotton
Received: {"Reference Date": "2024-03-04", "Observed": 98.44000244140625, "Next Week Prediction": 92.33294961293538}
Observed Prices in Rolling Window: [417.5, 1147.0, 98.44000244140625]
Cumulative Sum: 1662.9400024414062
Sent decision to Server 2 for commodity cotton: Sell
Sent:     coffee
Received: {"Reference Date": "2024-0

,Commodity,Action,Observed Price,Predicted Price,Date
0,corn,Buy,417.500000,484.468750,2024-03-05 14:08:02
1,soybean,Sell,1147.000000,1275.982500,2024-03-05 14:08:34
2,cotton,Sell,98.440002,92.332950,2024-03-05 14:08:35
3,coffee,Sell,197.399994,187.756333,2024-03-05 14:08:36
